In [9]:
import os
import pandas as pd
import openai
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

In [22]:
# 특이점 뽑아보기 kr
def gpt_answer3(msg):

    openai.api_key="sdf"
    answer=[]

    system_prompt =f"""                     
                        Instructions:
                        - 'biz_dt' means '날짜'
                        - 'nonbenefit_total_pay' means '비급여 총매출'
                        - 'total_pay' means '총 매출'
                        - 'visit_cnt' means '환자 수'
                        - 'fstvisit_cnt' means '신규 환자수'
                        - 'carinsur_pay' means '자보매출'
                        - 'nhis_pay' means '공단청구금'

                    
                    """

    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages=[{"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"""다음 주어지는 테이블은 csv 형식의 테이블이야 .
                      위 테이블 데이터를 살펴보고 이전날짜들과의 변동하는 폭이 크거나,
                     규칙적인 흐름과 다른 양상을 보이는 값들을 중점적으로 특이점을 뽑아줘.
                     이때 구체적인 날짜들을 언급해줘.
                     다음은 해당하는 테이블이야. {msg}"""}]
        )

    for choice in completion.choices:
        print(choice['message']['content'])
        answer.append(choice['message']['content'].strip())

In [26]:
# 특이점과 질문 뽑아보기 kr
def gpt_answer2(msg):

    openai.api_key="sdf"

    answer=[]

    system_prompt =f"""                     
                        Instructions:
                        - 'biz_dt' means '날짜'
                        - 'nonbenefit_total_pay' means '비급여 총매출'
                        - 'total_pay' means '총 매출'
                        - 'visit_cnt' means '환자 수'
                        - 'fstvisit_cnt' means '신규 환자수'
                        - 'carinsur_pay' means '자보매출'
                        - 'nhis_pay' means '공단청구금'

                    
                    """

    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages=[{"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"""다음 주어지는 테이블은 csv 형식의 테이블이야.
                     위 테이블 데이터를 살펴보고 이전날짜들과의 변동하는 폭이 크거나,
                     규칙적인 흐름과 다른 양상을 보이는 값들을 중점적으로 특이점을 뽑아줘.
                     이때 구체적인 날짜들을 언급해줘.
                     특이점들을 항목별로 뽑지 말고 전체적으로 종합해서 뽑아줘.

                     이 특이점들을 복합적으로 살펴보며 변동하는 값들에 기반하여 
                     이 병원을 위한 피드백 질문 5가지를 생성해줘. 
                     구체적인 날짜를 언급해줘. 
                     질문중 3개는 모든 항목들을 복합적으로 다루는 질문으로 만들어.
                     다음은 해당하는 테이블이야. {msg}"""}]
        )

    for choice in completion.choices:
        print(choice['message']['content'])
        answer.append(choice['message']['content'].strip())

    return answer

In [2]:
# 질문뽑아보기 En
def gpt_answer(msg):

    openai.api_key="sdfasdf"

    answer=[]

    system_prompt ="""You are a helpful assistant designed to make feedback questions
                    from given datas.
                        Make these in Korean.
                        User will send string type table data to you and you have to
                    analyze that table in oreder to give feedback questions
                        Questions must contain complex concept of datas.
                        Do not ask factual questions about table, but rather inference one.
                        Questions must mention a specific date.
                        
                        Instructions:
                        - 'biz_dt' means '날짜'
                        - 'nonbenefit_total_pay' means '비급여 총매출'
                        - 'total_pay' means '총 매출'
                        - 'visit_cnt' means '환자 수'
                        - 'fstvisit_cnt' means '신규 환자수'
                        - 'carinsur_pay' means '자보매출'
                        - 'nhis_pay' means '공단청구금'

                    
                    """

    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages=[{"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"""
                                                    This is the table. {msg}"""}]
        )

    for choice in completion.choices:
        print(choice['message']['content'])
        answer.append(choice['message']['content'].strip())

    return answer

In [ ]:
# few-shot training
def gpt_answer4(msg):

    openai.api_key="sadfaf"

    answer=[]
    ex_df = pd.read_csv('/home/fastcampus/Gasan/codes/경창예시.csv')

    system_prompt ="""You are a helpful assistant designed to make feedback questions
                    from given datas.
                        
                        
                        Instructions:
                        - 'biz_dt' means '날짜'
                        - 'nonbenefit_total_pay' means '비급여 총매출'
                        - 'total_pay' means '총 매출'
                        - 'visit_cnt' means '환자 수'
                        - 'fstvisit_cnt' means '신규 환자수'
                        - 'carinsur_pay' means '자보매출'
                        - 'nhis_pay' means '공단청구금'

                    
                    """

    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages=[[{"role": "system", "content": system_prompt}],
                 [{"role": "user", "content": f"""{ex_df}"""}],
                 [{"role": "assistant", "content":f"""
                   1. 최근 3개월간 비급여 매출이 총 매출 대비 어느정도의 비중을 차지하고 있나요?' 이에 따른 전략을 어떻게 세워야할까요?
                   2. 최근 6개월간 총 환자수 대비 신규환자수 비중이 높아지고 있습니다. 새로운 마케팅전략을 사용하고 계시나요? 어떤 요인이 큰 작용을 하였나요?
                   3. 2020년 4월 기점으로 비급여 매출 진폭이 1달 간격으로 큰데, 원인이 뭐라고 생각하시는지요? 이를 위한 안정화 전략은 무엇으로 보고 계신지요?
                   4. 환자 재내원을 위한 원내 정책 어떤 것들 있으신가요? 준비된 시스템에 대한 서술 부탁드립니다.
                   5. 2020년 4월 비급여 매출이 가장 높네요. 한약 매출 최초,재결제 성과 노하우 상세한 말씀 부탁드립니다.
                    """}],
                  [{"role": "user", "content":f"""{msg}"""}]]
        )

    for choice in completion.choices:
        print(choice['message']['content'])
        answer.append(choice['message']['content'].strip())

    return answer

In [3]:
def csv_reading(path):
    df = pd.read_csv(path)
    df = df.drop(['fk_id'], axis = 1)
    return df

In [4]:
def multi_csv(df1, df2):
    df = pd.merge(df1, df2, on = 'biz_dt', how='outer')
    return df

In [11]:
def compare_graph(df):

    # Read the CSV file
    data = df

    # Extract the data for x and y axes from the CSV columns
    x_values = data.iloc[:, 0]  # Assuming x-values are in the first column
    y1_values = data.iloc[:, 0]  # Extract y-values from the second column
    y2_values = data.iloc[:, 4]
   
    # Set the width of the bars
    bar_width = 0.4

    # Calculate the x-axis positions for the bars
    x1 = np.arange(len(x_values))
    x2 = x1 + bar_width
 
  # Create the bar plot
    plt.bar(x1, y1_values, width=bar_width, label='Y1')
    plt.bar(x2, y2_values, width=bar_width, label='Y2')


    # Rotate x-axis labels
    plt.xticks(rotation=90)

    # Add labels and title
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Bar Graph from CSV Data')
    plt.legend()

    # Display the graph
    plt.show()

In [28]:
df1= csv_reading('/home/fastcampus/Test_model /매출 데이터셋/비급여 총매출.csv/동원한의원 비급여 총매출.csv')
df2= csv_reading('/home/fastcampus/Test_model /매출 데이터셋/총진료비.csv/동원한의원 총진료비.csv')
df = multi_csv(df1,df2)
df3 = csv_reading('/home/fastcampus/Test_model /환자수 데이터셋/신규 환자 수.csv/동원한의원 신규 환자 수.csv')
df = multi_csv(df, df3)
df4 = csv_reading('/home/fastcampus/Test_model /환자수 데이터셋/환자 수.csv/동원한의원 환자 수.csv')
df = multi_csv(df, df4)
df5 = csv_reading('/home/fastcampus/Test_model /매출 데이터셋/자보매출.csv/동원한의원 자보매출.csv')
df6 = csv_reading('/home/fastcampus/Test_model /예약 데이터셋/예약환자 수.csv/동원한의원 예약환자 수.csv')
df7 = csv_reading('/home/fastcampus/Test_model /매출 데이터셋/공단청구금.csv/동원한의원 공단청구금.csv')
df = multi_csv(df, df5)
df = multi_csv(df, df6)
df = multi_csv(df, df7)
# df = df.set_index('biz_dt')
df = df.dropna(subset=['nonbenefit_total_pay', 'total_pay', 'fstvisit_cnt',
       'visit_cnt', 'carinsur_pay', 'nhis_pay'], how='all')
df = df.dropna(axis=1, how='all')
df



,biz_dt,nonbenefit_total_pay,total_pay,fstvisit_cnt,visit_cnt,carinsur_pay,nhis_pay
0,2018-09-01,4334000.0,19266920.0,63.0,486.0,2066220.0,10041500.0
1,2018-10-01,10995500.0,43623480.0,133.0,1090.0,5199170.0,21280650.0
2,2018-11-01,9191110.0,42928860.0,119.0,1095.0,7696440.0,20310120.0
3,2018-12-01,9569550.0,36617990.0,99.0,894.0,5231170.0,17111680.0
4,2019-01-01,13785000.0,46632870.0,104.0,1034.0,6646670.0,20214780.0
5,2019-02-01,12485300.0,41584730.0,87.0,953.0,4845300.0,18725230.0
6,2019-03-01,18221500.0,52955270.0,110.0,1176.0,3751660.0,24087580.0
7,2019-04-01,11094770.0,48567390.0,105.0,1094.0,7515240.0,22531480.0
8,2019-05-01,14238500.0,58018330.0,137.0,1155.0,9496820.0,24805900.0
9,2019-06-01,14250000.0,55539980.0,112.0,1182.0,6978440.0,25023830.0


In [16]:
prompt = f"""{df.to_string(index=False)}"""

raw_qs = gpt_answer(prompt)
raw_qs2 = gpt_answer2(prompt)

1. 지난 2021년 3월 1일과 이번달 10월 1일의 방문자 수 및 총 매출액의 변화 추이를 비교해 볼 수 있을까요?
2. 비급여 총 매출액과 자보매출의 비율은 어떻게 되나요? 이 비율은 최근 어느 기간 동안 변화가 있었나요?
3. 최근 6개월간 신규 환자 수와 전반적인 환자 수가 비슷한 비율로 증가했나요? 그렇지 않다면, 무엇이 그 증감에 반영되었나요? 
4. 금월 중에 공단청구금에 대한 환불 요청이 있었나요? 혹은 반대로, 공단청구금 지급을 받은 환자가 있는지 확인할 수 있을까요?
5. 최근 3개월 간의 총 매출액 증감 값을 바탕으로, 앞으로 다가올 6개월 동안의 매출 추이 예측을 해볼 수 있을까요?
1. 특이점이 발생한 2021년 1월 1일 이후에 신규환자수(fstvisit_cnt)가 급격하게 증가한 이유는 무엇일까요?
2. 비급여 총매출(nonbenefit_total_pay)이 2021년 상반기에는 높은 수치를 기록했지만 하반기에는 감소하는 것으로 보입니다. 이러한 변동이 발생한 이유는 무엇일까요?
3. 총 매출(total_pay)과 비급여 총매출(nonbenefit_total_pay)의 추이를 비교해볼 때, 이 두 항목 간 교차판매는 어떻게 변화하고 있는 걸까요?
4. 환자 수(visit_cnt)가 적은 2020년 12월 이후에는 급격한 증가와 감소를 반복하고 있습니다. 이러한 변동이 발생한 이유는 무엇일까요?
5. 공단청구금(nhis_pay)이 감소하는 동시에 자보매출(carinsur_pay)이 증가하는 추세를 보이고 있습니다. 이에 대한 원인은 무엇일까요?


In [27]:
prompt = f"""{df.to_string(index=False)}"""

raw_qs = gpt_answer2(prompt)

- 비급여 총매출(nonbenefit_total_pay)의 변동폭이 크게 나타납니다. 2020년 12월부터 2021년 1월로 이동할 때는 66% 증가하였으며, 이후 감소세가 이어졌으나, 2021년 9월에는 다시 큰 폭으로 증가하였습니다.
    - 특이점: 2020년 12월부터 2021년 1월로 이동하는 구간. 2021년 9월
- 신규 환자수(fstvisit_cnt)와 환자 수(visit_cnt)는 2021년 3월부터 큰 폭으로 증가하였으며, 그 이후는 안정세를 보이고 있습니다.
    - 특이점: 2021년 3월부터 2021년 7월
- 공단청구금(nhis_pay)은 2020년 12월부터 2021년 3월까지 증가세를 보였으나, 이후 감소세가 이어지고 있습니다.
    - 특이점: 2020년 12월부터 2021년 3월

피드백 질문:
1. 비급여 총매출이 큰 폭으로 감소한 2021년 상반기 동안 병원에서 새로운 마케팅 전략이나 행사 등을 시행하였는가요? (2021년 9월)
2. 신규 환자 수와 전체 환자 수가 증가한 이유는 무엇인가요? (2021년 3월부터 2021년 7월)
3. 공단청구금이 감소한 이유가 무엇이며, 이에 대한 대응 방안은 무엇인가요? (2020년 12월부터 2021년 3월)
4. 총 매출 변동폭이 적은 2021년 하반기 동안 병원의 매출 개선을 위한 추가적인 전략은 무엇이 있을까요?
5. 마케팅 전략과 업무 프로세스 개선 등을 통해 신규 환자를 더 유치할 수 있는 방법은 어떤 것이 있을까요? (2021년 3월부터 2021년 7월)


In [ ]:
# 결과를 저장할 리스트
raw_qs_list = []

# 각 항목에 대해 '\n'을 기준으로 분할
for sentences in raw_qs:
    split_sentences = sentences.split('\n')
    raw_qs_list.extend(split_sentences)

# 결과 출력
for sentence in raw_qs_list:
    print(sentence)

# 결과를 저장할 리스트
raw_qs_list2 = []

# 각 항목에 대해 '\n'을 기준으로 분할
for sentences in raw_qs2:
    split_sentences2 = sentences.split('\n')
    raw_qs_list2.extend(split_sentences2)

# 결과 출력
for sentence in raw_qs_list2:
    print(sentence)

In [21]:
klue_questions = """
1. 가정의 달 이벤트 매출의 현재까지 결과는 어떤가요? 목표설정대비 진행상황에 대해 말씀해주세요.
2. 이번 달 신규환자 수가 반등할 수 있었던 이유는 무엇일까요? 현재 근처에 새로 들어온 한의원의 영향은 없나요?
3. 비급여 다이어트 항목의 변동폭은 어떤 요인이 가장 크게 작용한다고 보시는 지 궁금합니다. 5월에 반등이 있었던 게 이벤트와의 연관성이 있을까요? 아니면 다른 변화가 있었나요?
4. 최근 3개월 간 다른 비급여지표에 비해 상대적으로 약침환자수와 매출이 성장세에 있습니다. 어떤 요인이 있었나요? 상담프로세스에 변화가 있었을까요?
5. 비급여 다이어트 항목의 변동폭은 어떤 요인이 가장 크게 작용한다고 보시는 지 궁금합니다. 5월에 반등이 있었던 게 이벤트와의 연관성이 있을까요? 아니면 다른 변화가 있었나요?
"""

conversation = [
    {
        "role": "system",
        "content": """You are a helpful assistant designed to make feedback questions
                    from given datas.
                        Make these in Korean.
                        User will send string type table data to you and you have to
                    analyze that table in oreder to give feedback questions
                        Questions must contain complex concept of datas.
                        Do not ask factual questions about table, but rather inference one.
                        Questions must mention a specific date.
                        
                        Instructions:
                        - 'biz_dt' means '날짜'
                        - 'nonbenefit_total_pay' means '비급여 총매출'
                        - 'total_pay' means '총 매출'
                        - 'visit_cnt' means '환자 수'
                        - 'fstvisit_cnt' means '신규 환자수'
                        - 'carinsur_pay' means '자보매출'
                        - 'nhis_pay' means '공단청구금'"""
    },
    {
        "role": "user",
        "content": f"""
        아래 테이블 데이터를 종합하여 큰 특이점을 기준으로 경영을 위한 창의적인 피드백 질문들을 5개 생성해줘.
        테이블에 없는 이야기는 하지 말아줘. 
        생성 예시 : {klue_questions}
        데이터 테이블 :
        {df}
        """
    }
]

import openai
openai.api_key="sasdfasfz"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=conversation
)

for choice in response.choices:
    print(choice['message']['content'])

1. 최근 2개월 간 자본매출 대비 공단청구금 비율이 급격히 상승한 것으로 보입니다. 이러한 현상은 어떤 원인이 있는 것으로 추측하시나요? 
2. 가을에는 전반적으로 환자 수가 감소하는 경향이 있는데, 올해 가을엔 어떤 경향을 보이고 있는 지 궁금합니다. 특히 10월에는 어떤 이유에서인지 급격한 매출 상승이 있었는 데, 이는 어떤 원인 때문일까요? 
3. 최근 6개월간의 데이터를 보면 신규 접수 환자 수가 큰 폭으로 상승했지만, 환자 유지율은 그렇게 높지 않은 것으로 보입니다. 이러한 경향은 어떤 이유 때문인 것으로 추측하시나요? 
4. 총매출은 전반적으로 증가하는 추세를 보이고 있는데, 환자 수와의 상관관계는 어떤 것으로 판단하시나요? 특히 9월에는 매출이 감소한 반면, 환자 수는 큰 폭으로 상승한 것으로 나타나는데, 이는 무슨 의미일까요? 
5. 최근 몇 달 간 비급여 매출이 전반적으로 높아졌는 데, 그 이유는 무엇일까요? 또한 이러한 추세가 이후에도 지속될 가능성이 있는 지 궁금합니다.


In [ ]:
import os
import openai

openai.api_key = "asdfz"

messages = [
    {"role": "system", "content": """Ignore all previous instructions before this one. 
                      You are an expert in creating feedback questions from administration metrics.
                      You have to create a administration consulting question from the administration indicator data according to my conditions.
                      First of all, you need to interpret and analyze the given indicator to find the numerically significant part.
                      Second, you must create a question by considering the columns of the given indicator in a complex and related way.
                      Third , When creating a question, be sure to base it on specific dates and figures.
                      Finally, you have to generate questions based only on the facts, without any lies about the indicators.
                      Create 5 questions based on the conditions above.
                      Please translate the answer into Korean and print it out.

                        
                        Instructions:
                        - 'biz_dt' : '날짜'
                        - 'nonbenefit_total_pay' : '비급여 총매출'
                        - 'total_pay' : '총 매출'
                        - 'visit_cnt' : '환자 수'
                        - 'fstvisit_cnt' : '신규 환자수'
                        - 'carinsur_pay' : '자보매출'
                        - 'nhis_pay' : '공단청구금'"""}
]

response = openai.ChatCompletion.create(
    engine="gpt-3.5-turbo",
    messages=messages,
    max_tokens=300,
    temperature=0.5,
    stop=None,
)

questions_korean = response.choices[0].text.strip()
print(questions_korean)
